In [1]:
import pandas as  pd 
import numpy as  np 
from datetime import datetime

import os
import matplotlib.pyplot as plt
from sklearn import  metrics, model_selection
from xgboost.sklearn import XGBClassifier


In [2]:
train_Numerical_data=pd.read_csv("all/train_Numerical_data.csv")
test=pd.read_csv("all/test.csv")


In [3]:
test["Store_id"]=test["Id"].apply( lambda x : x.split("|")[0])
test["Item_id"]=test["Id"].apply( lambda x : x.split("|")[1])
test["Date"]=test["Id"].apply( lambda x : x.split("|")[2])

In [4]:
test.Date=pd.to_datetime(test.Date).dt.date
train_Numerical_data.Date=pd.to_datetime(train_Numerical_data.Date).dt.date
train_Numerical_data[train_Numerical_data.Date.isin(test.Date)]

,Store_id,Item_id,Date,Quantity,Price,Net_Price,Net_Amount,Discount_Amount,Cost_Amount,Customer_Discount,Total_Discount,Periodic_Discount,Refund Qty_,date


In [5]:
start_date=datetime(2017,1,1).date()
end_date=datetime(2018,10,17).date()

In [6]:
item_to_use=test.Item_id.unique()
store_to_use=test.Store_id.unique()
train_Numerical_data=train_Numerical_data[train_Numerical_data.Item_id.isin(item_to_use)]
train_Numerical_data=train_Numerical_data[train_Numerical_data.Store_id.isin(store_to_use)]
train_Numerical_data=train_Numerical_data[train_Numerical_data.Quantity>0]


In [7]:
train_Numerical_data["date"]=pd.to_datetime(train_Numerical_data.Date)
train_Numerical_data["Date"]=train_Numerical_data.date.dt.date
train_Numerical_data=train_Numerical_data[train_Numerical_data.Date>=start_date]
# get firt date of item store
train_Numerical_data.sort_values(["Item_id","Store_id","Date"],inplace=True)
test["Date"]=pd.to_datetime(test.Date).dt.date


In [8]:
train_Numerical_data.Date.max()

datetime.date(2018, 10, 17)

In [9]:
test_keys=test.groupby(["Store_id"]).Item_id.unique().rename("list_of_uniqe_itmes").reset_index()
train_Numerical_data.sort_values(["Item_id","Store_id","Date"],inplace=True)
keys=pd.concat([train_Numerical_data,test]).sort_values(["Item_id","Store_id","Date"]).drop_duplicates(["Item_id","Store_id"])[["Item_id","Store_id","Date"]]

In [10]:
keys_to_use=[] 
for store , list_items in zip(test_keys.Store_id ,test_keys.list_of_uniqe_itmes): 
    keys_to_use.append(keys[(keys.Store_id==store)&(keys.Item_id.isin(list_items))])
keys=pd.concat(keys_to_use)
def create_data(store,start_date=start_date,end_date=end_date):
    print(store)
    frame=pd.DataFrame()
    keys_store=keys[keys.Store_id==store]
    items=keys_store.Item_id.values
    min_date=keys_store.Date.values
    Frames=[]
    for  item , date  in  zip(items,min_date) :
        frame=pd.DataFrame()
#         frame["Date"]=pd.date_range(date if start_date< date else start_date ,end_date).date
        frame["Date"]=pd.date_range(date ,end_date).date

        frame["Store_id"]=store
        frame["Item_id"]=item
        Frames.append(frame)
        del frame
    Frames=pd.concat(Frames)
    
    return Frames
full_data=pd.concat([ create_data(store) for store in keys.Store_id.unique()  ])

full_data.head()

S_0
S_1
S_10
S_11
S_12
S_13
S_14
S_15
S_16
S_17
S_18
S_19
S_2
S_20
S_21
S_22
S_3
S_4
S_5
S_6
S_7
S_8
S_9


,Date,Store_id,Item_id
0,2018-03-16,S_0,Item_1008
1,2018-03-17,S_0,Item_1008
2,2018-03-18,S_0,Item_1008
3,2018-03-19,S_0,Item_1008
4,2018-03-20,S_0,Item_1008


In [11]:
full_data["date"]=pd.to_datetime(full_data.Date)
full_data["day_of_week"]=full_data["date"].dt.dayofweek
full_data["year"]=full_data["date"].dt.year
full_data["woy"]=full_data["date"].dt.weekofyear
full_data["month"]=full_data["date"].dt.month
full_data["quarter"]=full_data["date"].dt.quarter
full_data["month_day"]=full_data["date"].dt.day
del full_data["date"] 

In [12]:
target=train_Numerical_data[train_Numerical_data.Quantity>0][["Date","Store_id","Item_id","Quantity"]]
train_target=target.groupby(["Date","Store_id","Item_id"]).Quantity.sum().rename("Quantity").reset_index()
train_target["Date"]=pd.to_datetime(train_target.Date).dt.date
full_data=full_data.merge(train_target,on=["Date","Store_id","Item_id"],how="left")
full_data.fillna(0,inplace=True)
full_data.loc[full_data.Date.isin(test.Date),"Quantity"]=np.nan
del train_target

### feature engineering

### Model 

In [13]:
test.Date=pd.to_datetime(test.Date).dt.date
Train=full_data[~full_data.Date.isin(test.Date)]
Test=full_data[full_data.Date.isin(test.Date)]
len(Test)-len(test)

0

### Train is the the final data to use for train your model good luck :) 

In [14]:
Test.columns

Index(['Date', 'Store_id', 'Item_id', 'day_of_week', 'year', 'woy', 'month',
       'quarter', 'month_day', 'Quantity'],
      dtype='object')

In [15]:
to_remove=['Date', 'Store_id', 'Item_id',"year","Quantity"]
from sklearn.model_selection import train_test_split
Train,Val=train_test_split(Train,test_size=0.1)

In [ ]:
### Model to use  

Train['Quantity'],class_names = pd.factorize(Train['Quantity'])

Train['Date'],_ = pd.factorize(Train['Date'])
Train['Store_id'],_ = pd.factorize(Train['Store_id'])
Train['Item_id'],_ = pd.factorize(Train['Item_id'])
Train['day_of_week'],_ = pd.factorize(Train['day_of_week'])
Train['year'],_ = pd.factorize(Train['year'])
Train['woy'],_ = pd.factorize(Train['woy'])
Train['month'],_ = pd.factorize(Train['month'])
Train['quarter'],_ = pd.factorize(Train['quarter'])
Train['month_day'],_ = pd.factorize(Train['month_day'])
Train['Quantity'],_ = pd.factorize(Train['Quantity'])
Train.head()



,Date,Store_id,Item_id,day_of_week,year,woy,month,quarter,month_day,Quantity
3160573,0,0,0,0,0,0,0,0,0,0
2206530,1,1,1,0,0,1,1,1,1,1
3241856,2,2,2,1,0,2,2,0,2,2
501331,3,3,3,1,1,3,3,2,1,2
897421,4,4,4,1,0,4,4,3,3,2


In [ ]:
X = Train.iloc[:,:-1]
y = Train.iloc[:,-1]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=123)
params = {
    'objective': 'binary:logistic',
    'max_depth': 2,
    'learning_rate': 1.0,
    'silent': 1.0,
    'n_estimators': 5
}

model = XGBClassifier(**params).fit(X_train, y_train)

In [ ]:
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
# how did our model perform?
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

### Create  Submission 

In [ ]:
Test["Quantity"]=p_test 
Test["Quantity"]=Test["Quantity"].apply( lambda x : x if x>0 else 0 )
test.Date=pd.to_datetime(test.Date)
Test.Date=pd.to_datetime(Test.Date)
del test["Quantity"]
test=test.merge(Test[["Quantity","Store_id","Item_id","Date"]],how="left",on=["Store_id","Item_id","Date"])
test[["Id","Quantity"]].to_csv("sub.csv",index=False)